In [1]:
import cv2
import skimage.io
import numpy as np
from helpers import *
from classes import *
from tkinter import *
    
img = skimage.io.imread('dock.jpg')
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

root = Tk()
wScreen, hScreen = root.winfo_screenwidth(), root.winfo_screenheight()
root.destroy()

color = (255,255,255)
reticleSize = 50
thickness = 3

fixationDemo = True
liveCaptureDemo = True

In [2]:
if fixationDemo:
    R = Retina('retina.pkl')
    R.calibrate(img)

    C = Cortex('cortex.pkl')
    C.calibrate(R)

    targetSize = np.array([hScreen//2.1,wScreen//2.1])
    captureSize = np.array(img.shape[:2])

    captureScale = max(targetSize)/max(captureSize)
    retinaScale = max(targetSize)/max(R.size)
    cortexScale = max(targetSize)/max(C.size)

    absoluteSizes = {'capture': (captureScale*captureSize).astype('int'),
                     'retina' : (retinaScale*R.size).astype('int'),
                     'cortex' : (cortexScale*C.size[::-1]).astype('int')}

    n = 100
    r = 0.55 * R.input_resolution[0]
    xs = np.array([int(np.cos(2*np.pi/n*x)*r) for x in range(0,n+1)]) + R.input_resolution[1]//2
    ys = np.array([int(np.sin(2*np.pi/n*x)*r) for x in range(0,n+1)]) + R.input_resolution[0]//2

    i = 0
    while True:
        R.setFixation(xs[i],ys[i])
        retinaScale = max(targetSize)/max(R.size)
        absoluteSizes['retina'] = (retinaScale*R.size).astype('int')
        R.sample(img)
        R.backProject()
        C.backProject(R.getResult())

        modifiedInput = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        modifiedInput = cv2.circle(modifiedInput,(xs[i],ys[i]),R.size.max()//2,color,thickness)
        modifiedInput = cv2.line(modifiedInput, (xs[i],ys[i]-50), (xs[i],ys[i]+50), color, thickness)
        modifiedInput = cv2.line(modifiedInput, (xs[i]-50,ys[i]), (xs[i]+50,ys[i]), color, thickness)

        cv2.imshow('Input',resize(modifiedInput, absoluteSizes['capture']))
        cv2.imshow('Back Projected',resize(get_retinaBackProjected_BGR(R), absoluteSizes['retina']))
        cv2.imshow('Cortical Image',resize(get_cortexBackProjected_BGR(C), absoluteSizes['cortex']))
        key = cv2.waitKey(1)
        if key == 27: #esc
                break
        i+=1
        if i==len(xs):
            i=0
    cv2.destroyAllWindows()

Exception: Could not find previously saved arrays! Please make sure to initlaize them.

In [ ]:
if liveCaptureDemo:
    cap = cv2.VideoCapture(0)
    
    #Define capture resolution
    #If the actual set values are not supported by the camera,
    #highest supported values below the entered numbers is selected
    cap.set(3, 1920)
    cap.set(4, 1080)
    ret, calibration_image = cap.read()
    calibration_image = cv2.cvtColor(calibration_image, cv2.COLOR_BGR2GRAY)
    print("Camera resolution was set to %ix%i" %calibration_image.shape[::-1])
    
    R = Retina('retina.pkl')
    R.calibrate(calibration_image)

    C = Cortex('cortex.pkl')
    C.calibrate(R)
    
    x, y = R.fixation
    targetSize = np.array([hScreen//2.1,wScreen//2.1])
    captureSize = np.array(img.shape[:2])

    captureScale = max(targetSize)/max(captureSize)
    retinaScale = max(targetSize)/max(R.size)
    cortexScale = max(targetSize)/max(C.size)

    absoluteSizes = {'capture': (captureScale*captureSize).astype('int'),
                     'retina' : (retinaScale*R.size).astype('int'),
                     'cortex' : (cortexScale*C.size[::-1]).astype('int')}
    
    try:
        while(cap.isOpened()):
            isValid, frame = cap.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            modifiedInput = np.copy(gray)
            modifiedInput = cv2.circle(modifiedInput,(x,y),R.size.max()//2,color,thickness)
            modifiedInput = cv2.line(modifiedInput, (x,y-50), (x,y+50), color, thickness)
            modifiedInput = cv2.line(modifiedInput, (x-50,y), (x+50,y), color, thickness)
            
            R.sample(gray)
            R.backProject()
            C.backProject(R.getResult())
            
            cv2.imshow('Capture',resize(modifiedInput, absoluteSizes['capture']))
            cv2.imshow('Back Projected',resize(get_retinaBackProjected_GRAY(R), absoluteSizes['retina']))
            cv2.imshow('Cortical Image',resize(get_cortexBackProjected_GRAY(C), absoluteSizes['cortex']))
            key = cv2.waitKey(1)
            if key == 27: #esc
                    break
    finally:
        cap.release()
        cv2.destroyAllWindows()